# **Agents and Tools**

### Nihar Lohar 
### BTech Artificial Intelligence, Mumbai


In [ ]:
!pip install pandas mistralai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 440.5/440.5 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.3/160.3 kB 16.2 MB/s eta 0:00:00


In [ ]:
import pandas as pd

data = {
    'transaction_id': ['T1001', 'T1002', 'T1003', 'T1004', 'T1005'],
    'customer_id': ['C001', 'C002', 'C003', 'C002', 'C001'],
    'payment_amount': [125.50, 89.99, 120.00, 54.30, 210.20],
    'payment_date': ['2021-10-05', '2021-10-06', '2021-10-07', '2021-10-05', '2021-10-08'],
    'payment_status': ['Paid', 'Unpaid', 'Paid', 'Paid', 'Pending']
}

df = pd.DataFrame(data)


In [ ]:
def retrieve_payment_status(df: data, transaction_id: str) -> str:
    if transaction_id in df.transaction_id.values:
      return json.dumps(
          {
              'status': df[df.transaction_id == transaction_id].payment_status.item()
          }
      )
    return json.dumps({'error': 'Transaction not found'})

def retrieve_payment_date(df: data, transaction_id: str) -> str:
    if transaction_id in df.transaction_id.values:
      return json.dumps(
          {
              'date': df[df.transaction_id == transaction_id].payment_date.item()
          }
      )
    return json.dumps({'error': 'Transaction not found'})
def retrieve_payment_amount(df: data, transaction_id: str) -> str:
    if transaction_id in df.transaction_id.values:
      return json.dumps(
          {
              'amount': df[df.transaction_id == transaction_id].payment_amount.item()
          }
      )
      return json.dumps({'error': 'Transaction not found'})


In [ ]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "retrieve_payment_status",
            "description": "Get payment status of a transaction",
            "parameters": {
                "type": "object",
                "properties": {
                    "transaction_id": {
                        "type": "string",
                        "description": "The transaction ID."
                    }
                },
                "required": ["transaction_id"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "retrieve_payment_date",
            "description": "Get payment date of a transaction",
            "parameters": {
                "type": "object",
                "properties": {
                    "transaction_id": {
                        "type": "string",
                        "description": "The transaction ID."
                    }
                },
                "required": ["transaction_id"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "retrieve_payment_amount",
            "description": "Get payment amount of a transaction",
            "parameters": {
                "type": "object",
                "properties": {
                    "transaction_id": {
                        "type": "string",
                        "description": "The transaction ID."
                    }
                },
                "required": ["transaction_id"],
            },
        },
    }
]


In [ ]:
import functools

names_to_functions = {
  'retrieve_payment_status': functools.partial(retrieve_payment_status, df=df),
  'retrieve_payment_date': functools.partial(retrieve_payment_date, df=df),
  'retrieve_payment_amount': functools.partial(retrieve_payment_amount, df=df)
}


In [ ]:
messages = [
    {
        "role": "user",
        "content": "What's the status of my transaction T1001."
    }
]

In [ ]:
import getpass
import os

os.environ["MISTRAL_API_KEY"] = getpass.getpass("Enter your Mistral API key: ")
api_key = os.environ["MISTRAL_API_KEY"]

Enter your Mistral API key: ··········


In [ ]:
from mistralai import Mistral

model = "mistral-small-2506"

client = Mistral(api_key=api_key)
response = client.chat.complete(
    model=model,
    messages=messages,
    tools=tools,
    tool_choice="any"
)

response

ChatCompletionResponse(id='723b7bb305c04c5f80c68d73b86a4b30', object='chat.completion', model='mistral-small-2506', usage=UsageInfo(prompt_tokens=226, completion_tokens=18, total_tokens=244, prompt_audio_seconds=Unset()), created=1758954865, choices=[ChatCompletionChoice(index=0, message=AssistantMessage(content='', tool_calls=[ToolCall(function=FunctionCall(name='retrieve_payment_status', arguments='{"transaction_id": "T1001"}'), id='55sbdVju4', type=None, index=0)], prefix=False, role='assistant'), finish_reason='tool_calls')])

In [ ]:
messages.append(response.choices[0].message)

In [ ]:
messages

[{'role': 'user', 'content': "What's the status of my transaction T1001."},
 AssistantMessage(content='', tool_calls=[ToolCall(function=FunctionCall(name='retrieve_payment_status', arguments='{"transaction_id": "T1001"}'), id='55sbdVju4', type=None, index=0)], prefix=False, role='assistant')]

In [ ]:
import json

tool_call = response.choices[0].message.tool_calls[0]

function_name = tool_call.function.name

function_params = json.loads(tool_call.function.arguments)

print("\nfunction_name: ", function_name, "\nfunction_params: ", function_params)


function_name:  retrieve_payment_status 
function_params:  {'transaction_id': 'T1001'}


In [ ]:
function_result = names_to_functions[function_name](**function_params)

function_result


'{"status": "Paid"}'

In [ ]:
messages.append(
    {
        "role":"tool",
        "name":function_name,
        "content":function_result,
        "tool_call_id":tool_call.id
    }
)

In [ ]:
messages

[{'role': 'user', 'content': "What's the status of my transaction T1001."},
 AssistantMessage(content='', tool_calls=[ToolCall(function=FunctionCall(name='retrieve_payment_status', arguments='{"transaction_id": "T1001"}'), id='55sbdVju4', type=None, index=0)], prefix=False, role='assistant'),
 {'role': 'tool',
  'name': 'retrieve_payment_status',
  'content': '{"status": "Paid"}',
  'tool_call_id': '55sbdVju4'}]

In [ ]:
response = client.chat.complete(
    model=model,
    messages=messages

)

response.choices[0].message.content

'The status of your transaction **T1001** is **Paid**.'

**Putting all in a function**

In [ ]:
import json

def get_response(user_query, client, model, tools, names_to_functions, system_prompt=None):
    messages = []
    if system_prompt:
        messages.append(
            {
                "role": "system",
                "content": system_prompt
            }
        )

    messages.append(
        {
            "role": "user",
            "content": user_query
        }
    )

    response = client.chat.complete(
        model=model,
        messages=messages,
        tools=tools,
        tool_choice="any"
    )

    messages.append(response.choices[0].message)

    tool_call = response.choices[0].message.tool_calls[0]
    function_name = tool_call.function.name
    function_params = json.loads(tool_call.function.arguments)

    function_result = names_to_functions[function_name](**function_params)

    messages.append(
        {
            "role": "tool",
            "name": function_name,
            "content": function_result,
            "tool_call_id": tool_call.id
        }
    )

    response = client.chat.complete(
        model=model,
        messages=messages
    )

    final_response = response.choices[0].message.content

    return final_response

In [ ]:
system_prompt = """
You are a helpful assistant that provides information about payment transactions.
Answer concisely.
"""

user_query = "What's the amount of my transaction T1001?"
print(f"User Query: {user_query}")

final_response = get_response(
    user_query, client, model, tools, names_to_functions, system_prompt=system_prompt
)
print(f"Final Model Response: {final_response}")

User Query: What's the amount of my transaction T1001?
Final Model Response: The amount of your transaction T1001 is $125.50.


In [ ]:
# Without system prompt
# user_query_2 = "What's the date of my transaction T1002?"
user_query_2 = "What's the status of my transaction T1001?"
print(f"\nUser Query: {user_query_2}")

final_response_2 = get_response (
    user_query_2, client, model, tools, names_to_functions
)
print(f"Final Model Response: {final_response_2}")


User Query: What's the status of my transaction T1001?
Final Model Response: The status of your transaction **T1001** is **Paid**.
